In [26]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg,max,min,sum, udf, col, when, count, round, concat_ws, lit, first
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField
from pyspark.sql import functions as F
from pyspark.sql import Window
import os
import unicodedata
import inspect

import sys
sys.path.insert(0, os.path.abspath("../src"))  # Adiciona src ao caminho

from funcao import remove_acentos

spark = SparkSession.builder.appName("NotasPorMateriaEAluno").getOrCreate()

In [30]:
remove_acentos_udf = udf(remove_acentos, StringType())

In [28]:
data = [("João", "Matemática", 8), ("João", "Matemática", 9), 
        ("Maria", "História", 7), ("Maria", "História", 6), 
        ("Pedro", "Ciências", 10), ("Pedro", "Ciências", 8),
        ("Ana", "Matemática", 9), ("Ana", "História", 8), 
        ("Ana", "Ciências", 7), ("Carlos", "Matemática", 6), 
        ("Carlos", "História", 5), ("Carlos", "Ciências", 4)]

df = spark.createDataFrame(data, ["aluno", "mat", "nota"])
df.show()

+------+----------+----+
| aluno|       mat|nota|
+------+----------+----+
|  João|Matemática|   8|
|  João|Matemática|   9|
| Maria|  História|   7|
| Maria|  História|   6|
| Pedro|  Ciências|  10|
| Pedro|  Ciências|   8|
|   Ana|Matemática|   9|
|   Ana|  História|   8|
|   Ana|  Ciências|   7|
|Carlos|Matemática|   6|
|Carlos|  História|   5|
|Carlos|  Ciências|   4|
+------+----------+----+



In [31]:

df_unifica_linhas = (
    df
    .withColumn("mat_sem_acentos", remove_acentos_udf(col("mat")))
    .groupBy(col("aluno"))
    .pivot("mat_sem_acentos")
    .agg(first(col("nota")))
    )

In [32]:
df_2 = (
    df_unifica_linhas
    .fillna(0, subset=["Matematica", "Historia", "Ciencias"])
)
df_2.show()

+------+--------+--------+----------+
| aluno|Ciencias|Historia|Matematica|
+------+--------+--------+----------+
|Carlos|       4|       5|         6|
|  João|       0|       0|         8|
| Pedro|      10|       0|         0|
|   Ana|       7|       8|         9|
| Maria|       0|       7|         0|
+------+--------+--------+----------+

